<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, deploy, and score a PMML model to predict species of iris</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to store a sample Predictive Model Markup Language (PMML) model and score test data. You will use the Iris data set to predict the species of an iris flower. This data set contains measurements of the iris perianth flower. 

Some familiarity with Python is helpful. This notebook is compatible Watson Studio Desktop 1.1, Watson Machine Learning Server 1.1, and Python 3.6.


## Learning goals

You will learn how to:

-  Set up the Watson Machine Learning (WML) Server.
-  Deploy a PMML model.
-  Score the deployed model.


## Table of Contents

1.	[Set up the environment](#setup)<br>
    1.1 [ Connecting to Watson Studio Desktop](#wsd) <br>
    1.2 [Connecting to Watson Machine Learning Server](#wmls)<br>
2.	[Save a PMML model](#save)<br>
    2.1 [Persist a PMML model](#persist)<br>
3. [Create an online deployment](#deploy)<br>
    3.1 [Score a test data record](#score)<br>
4.	[Summary and next steps](#summary)

To get started on Watson Machine Learning (WML) Server, find documentation on installion and set up <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-install-over.html" target="_blank" rel="noopener no referrer">here</a>.

<a id="setup"></a>
## 1. Set up the environment

In this section, you will learn how to use the python client to connect to both Watson Studio Desktop and Watson Machine Learning Server. If you only intend to save the model on WSD, you will need to follow the steps in Section 1.1: [Connecting to Watson Studio Desktop](#wsd).

If you want to use the WML Server, you'll need to refer to Section 1.2: [Connecting to Watson Machine Learning Server](#wmls). From there you'll be able to save, deploy, and score the model in your deployment space on the server.

First, import the `watson-machine-learning-client` module and authenticate the service instance.

In [ ]:
!pip install --upgrade watson-machine-learning-client-V4==1.0.60

In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

### 1.1 Connecting to Watson Studio Desktop <a id="wsd"></a>

To associate the python client with Watson Studio Desktop, use the following credentials.

In [3]:
from project_lib.utils import environment
url = environment.get_common_api_url()

wml_credentials = {
     "instance_id": "wsd_local",
     "url": url,
     "version": "1.1"
}

Now, instantiate a `WatsonMachineLearningAPIClient` object.

In [4]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [5]:
client.version

'1.0.60'

Setting the default project is mandatory when you use WSD. You can use the cell below.

In [6]:
from project_lib import Project
project = Project.access()
project_id = project.get_metadata()["metadata"]["guid"]

client.set.default_project(project_id)

'SUCCESS'

To proceed, you can go directly to the Section 2: [Save a PMML model](#save).

<a id="wmls"></a>
### 1.2 Connecting to Watson Machine Learning Server <a id="wmls"></a>

In this subsection, you will learn how to set up the Watson Machine learning (WML) Server that is required in order to save, deploy, and score a PMML model in the Watson Machine learning (WML) repository.

**Connect to the Watson Machine Learning Server using the Python client**<br><br>

<div class="alert-block alert-info"><br>
To install the Watson Machine Learning Server, follow <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-install-over.html" target="_blank" rel="noopener no referrer">these documentation steps</a>. To connect to the WML server and find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/wmls-connect.html" target="_blank" rel="noopener no referrer">Documentation</a>.<br><br>
</div>

**Action**: Enter your WML Server credentials in the following cell.

In [13]:
# Enter your credentials here.
wml_credentials = {
    "url": "<URL>:31843",
    "username": "---",
    "password": "---",
    "instance_id": "wml_local",
    "version": "1.1"
}

Now, instantiate a `WatsonMachineLearningAPIClient` object.

In [15]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [16]:
client.version

'1.0.60'

Since you're using WML Server in this section, you can obtain the space UID by using the following cells.

<div class="alert-block alert-info"><br>
You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Deployment Spaces</b> from the Navigation Menu on the top left of this page.<br><br></div>

Alternatively you can create a deployment and obtain its UID using the code in the following cell. The cell is not executable cell at this stage, but you can enter the name of your space in the metadata and use it if needed.

In [17]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    instance_details = client.service_instance.get_details()
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')`.

In [18]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = 0da6514c-1e1e-46db-8fa8-b3d3cb9053ff


Setting the default space is mandatory when you use WML Server. You can set this using the cell below.

In [19]:
client.set.default_space(space_uid)

'SUCCESS'

<a id="save"></a>
## 2. Save a PMML model

In this section, you will use the Python API client to create an online deployment and score the PMML model with a new data record.

For more information, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">V4 Python client documentation</a>.


Use `wget` to download the sample PMML model, `iris_chaid.xml` from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.  

<div class="alert-block alert-info"><br>
You may need to install the <tt>wget</tt> package. To install the <tt>wget</tt> package, run the following command.<br><br></div>

In [ ]:
!pip install --upgrade wget

In [21]:
# Download the sample PMML model and the 'iris_chaid.xml' file from the Github repo.
import wget
import os

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store the downloaded file as *CHAID PMML model for Iris data*, and then list all the saved models. First, you need to create the model metadata.


In [22]:
# Model metadata
props_pmml = {
    client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
    client.repository.ModelMetaNames.RUNTIME_UID: 'pmml_4.2.1', 
    client.repository.ModelMetaNames.TYPE: 'pmml_4.2.1'
}

You can extract the model UID from the saved model details.

In [23]:
# Create the model artifact.
model_artifact = client.repository.store_model(filename, meta_props=props_pmml)
model_uid = client.repository.get_model_uid(model_artifact)
print("Model UID = " + model_uid)

Model UID = 90b9ba6f-debb-47cc-829f-81dc048e3586


Get the saved model metadata from WML using the model UID.

In [24]:
# Details about the model.
model_details = client.repository.get_details(model_uid)
from pprint import pprint
pprint(model_details)

{'entity': {'content_status': {'state': 'persisted'},
            'name': 'CHAID PMML model for Iris data',
            'runtime': {'href': '/v4/runtimes/pmml_4.2.1'},
            'space': {'href': '/v4/spaces/0da6514c-1e1e-46db-8fa8-b3d3cb9053ff'},
            'type': 'pmml_4.2.1'},
 'metadata': {'created_at': '2020-01-22T20:04:51.002Z',
              'guid': '90b9ba6f-debb-47cc-829f-81dc048e3586',
              'href': '/v4/models/90b9ba6f-debb-47cc-829f-81dc048e3586?space_id=0da6514c-1e1e-46db-8fa8-b3d3cb9053ff',
              'id': '90b9ba6f-debb-47cc-829f-81dc048e3586',
              'modified_at': '2020-01-22T20:04:52.002Z',
              'owner': '1000330999'}}


You can list all stored models using the `list_models` method.

In [25]:
# Display a list of all the models.
client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  ----------
GUID                                  NAME                            CREATED                   TYPE
90b9ba6f-debb-47cc-829f-81dc048e3586  CHAID PMML model for Iris data  2020-01-22T20:04:51.002Z  pmml_4.2.1
------------------------------------  ------------------------------  ------------------------  ----------


<div class="alert-block alert-info"><br>
From the list of stored models, you can see that model is successfully saved. <br>

With Watson Studio Desktop credentials, this means you've saved the model in your project. You can see the saved model in your project UI by clicking on your project name in the breadcrumb at the top of the application. <br>

With WML Server credentials, this means that you've saved the model in your deployment space. You can view your model by selecting <b>Deployment Spaces</b> from the Navigation Menu and clicking on your deployment space name.<br>
<br></div>

If you're using WML Server, proceed to Section 3: [Create an online deployment (WML Server only)](#deploy). If you're using Watson Studio Desktop, you may skip to the [summary](#summary).

## 3. Create an online deployment (WML Server only) <a id="deploy"></a>

Now, create an online deployment, *Iris species prediction*, for the stored model, and list all the online deployments in the deployment space. You'll create deployment metadata and use the model UID obtained in the previous section.


In [26]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Iris species prediction",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [27]:
# Create an online deployment.
deployment_details = client.deployments.create(model_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: '90b9ba6f-debb-47cc-829f-81dc048e3586' started

#######################################################################################


initializing..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='a3fb35c6-c4bb-4bbc-b955-f9dce47eddf0'
------------------------------------------------------------------------------------------------




In [28]:
# # List the deployments.
client.deployments.list()

------------------------------------  -----------------------  -----  ------------------------  -------------
GUID                                  NAME                     STATE  CREATED                   ARTIFACT_TYPE
a3fb35c6-c4bb-4bbc-b955-f9dce47eddf0  Iris species prediction  ready  2020-01-22T20:04:56.923Z  model
------------------------------------  -----------------------  -----  ------------------------  -------------


<div class="alert-block alert-info"><br>
From the list of deployed models in the deployment space, you can see that model was successfully created.<br><br></div>

Now, you can check details of your deployments.

In [29]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment UID = {}'.format(deployment_uid))

Deployment UID = a3fb35c6-c4bb-4bbc-b955-f9dce47eddf0


You can check the deployment details by running the following cell.

In [30]:
# Deployment details.
client.deployments.get_details(deployment_uid)

{'metadata': {'parent': {'href': ''},
  'guid': 'a3fb35c6-c4bb-4bbc-b955-f9dce47eddf0',
  'modified_at': '',
  'created_at': '2020-01-22T20:04:56.923Z',
  'href': '/v4/deployments/a3fb35c6-c4bb-4bbc-b955-f9dce47eddf0'},
 'entity': {'name': 'Iris species prediction',
  'custom': {},
  'online': {},
  'description': '',
  'space': {'href': '/v4/spaces/0da6514c-1e1e-46db-8fa8-b3d3cb9053ff'},
  'status': {'state': 'ready',
   'online_url': {'url': 'https://vml-wml-qa1.fyre.ibm.com:31843/v4/deployments/a3fb35c6-c4bb-4bbc-b955-f9dce47eddf0/predictions'}},
  'asset': {'href': '/v4/models/90b9ba6f-debb-47cc-829f-81dc048e3586?space_id=0da6514c-1e1e-46db-8fa8-b3d3cb9053ff'},
  'auto_redeploy': False}}

### 3.1 Score a test data record<a id="score"></a>

Now, you can score the data and predict the species of iris flower given from the data. Creat the scoring payload.

In [31]:
# Score data and predict species of flower. Prepare scoring payload.
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'],
        'values': [[5.1, 3.5, 1.4, 0.2]]
    }]
}
pprint(job_payload)

{'input_data': [{'fields': ['Sepal.Length',
                            'Sepal.Width',
                            'Petal.Length',
                            'Petal.Width'],
                 'values': [[5.1, 3.5, 1.4, 0.2]]}]}


In [32]:
# Score data and predict species of flower. 
predictions = client.deployments.score(deployment_uid, job_payload)
print(predictions)

{'predictions': [{'fields': ['$R-Species', '$RC-Species', '$RP-Species', '$RP-setosa', '$RP-versicolor', '$RP-virginica', '$RI-Species'], 'values': [['setosa', 1.0, 1.0, 1.0, 0.0, 0.0, '1']]}]}


As we can see from the prediction, this is a species of Iris Setosa.

<a id="summary"></a>
## 4. Summary and next steps     

You successfully completed this notebook! You learned how to save the PMML model both on Watson Studio Desktop and Watson Machine Learning Server. You can now deploy, and score the PMML model on WML Server as well.

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success">

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SS3PWM_1.0.0/wsj/wmls/overview.html" target="_blank" rel="noopener noreferrer">Watson Machine Learning Server</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 <li> <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener noreferrer">Sample models</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a><br></li>
 </div>

### Citation

Dua, D. and Karra Taniskidou, E. (2017). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


### Authors

**Wojciech Sobala** is a Data Scientist at IBM.  <br><br>
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2018, 2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>